This notebook demonstrates the one time pad as a set of 3 functions:


1.   Generate One Time Pad
2.   Use one time pad to encrypt message
3.   Decrypt message with One Time Pad


In [308]:
import collections
import numpy as np

In [506]:
def generateOTP(pad_size):
    rng = np.random.default_rng()
    encoder = {
        'A':0, 'B':1, 'C':2, 'D':3, 'E':4,'F':5, 'G':6, 'H':7, 'I':8, 'J':9,
        'K':10, 'L':11, 'M':12, 'N':13, 'O':14, 'P':15, 'Q':16, 'R':17, 'S':18,
        'T':19, 'U':20, 'V':21, 'W':22, 'X':23, 'Y':24, 'Z':25, ' ':26,
        '.':27, #',':28, '!':29, '?':30,'-':31,
    }
    decoder = {value: key for key, value in encoder.items()}

    otp_int = rng.choice(len(encoder), size=pad_size ,replace=True)
    otp_chars = [decoder[i] for i in otp_int]
    otp_string = ''.join(otp_chars)

    return otp_string

In [576]:
def encrypt_message(message_string, otp_string):
    encoder = collections.defaultdict(lambda:27)
    encoder.update({
        'A':0, 'B':1, 'C':2, 'D':3, 'E':4,'F':5, 'G':6, 'H':7, 'I':8, 'J':9,
        'K':10, 'L':11, 'M':12, 'N':13, 'O':14, 'P':15, 'Q':16, 'R':17, 'S':18,
        'T':19, 'U':20, 'V':21, 'W':22, 'X':23, 'Y':24, 'Z':25, ' ':26,
        '.':27,
    })
    decoder = {value: key for key, value in encoder.items()}

    message_string = message_string.upper().replace('\n','').replace('\t','')

    # Add XXX's to the end of the message to fully use one_time_pad
    filler_int = 23*np.ones(len(otp_string[len(message_string):]))
    filler_string = ''.join([decoder[i] for i in filler_int])
    message_string = message_string + filler_string

    message_int = [encoder[i] for i in message_string]
    message_int = message_int[0:len(otp_string)]

    otp_int = np.array([encoder[i] for i in otp_string])
    cipher_int = (otp_int[0:len(message_int)] + message_int) % len(decoder)
    # modulo (%) decoder: for each character not found in encoder,
    # its length increases.  decoder remains constant

    cipher_string = ''.join([decoder[i] for i in cipher_int])
    return cipher_string

In [559]:
def decrypt_message(cipher_string, otp_string):
    encoder = {
        'A':0, 'B':1, 'C':2, 'D':3, 'E':4,'F':5, 'G':6, 'H':7, 'I':8, 'J':9,
        'K':10, 'L':11, 'M':12, 'N':13, 'O':14, 'P':15, 'Q':16, 'R':17, 'S':18,
        'T':19, 'U':20, 'V':21, 'W':22, 'X':23, 'Y':24, 'Z':25, ' ':26,
        '.':27, #',':28, '!':29, '?':30,'-':31,
    }
    decoder = {value: key for key, value in encoder.items()}

    otp_int = np.array([encoder[i] for i in otp_string])
    cipher_int = [encoder[i] for i in cipher_string]
    decipher_int = (cipher_int - otp_int[0:len(cipher_string)]) % len(decoder)
    decipher_string = ''.join([decoder[i] for i in decipher_int])
    return decipher_string

# Sample: Multiline triple quote text

In [560]:
one_time_pad = generateOTP(200)
one_time_pad

'YZ  X.NRDMXDKUQUEEFSFCGDVLFRXSBTRTGEE OY.VLPNPJGSRKMAKZWTTLK OQ.KBDDTCULXBCIKTNS NPHFSCWONGNVLFKCJTQIEOMQOMHMVN.CXTVRLYQM.QIDVOVLCVI IVPRIGOZORD.UGKUQAVHJEZSHAOTXHSOEPJBWRHPZVZ.RNSRF WKHU .IOYKLAPCABR'

In [561]:
fake_pad = generateOTP(200)

In [562]:
message = """
    E equals mc squared !!
    The quick fox jumps over the lazy dog?
    Sphinx of black quartz hear my vow. &&&90
"""

In [566]:
cipher = encrypt_message(message, one_time_pad)
display(cipher)

'WXYY.ZRFXMGVIESSWUZSWGJBUKDPVQU VRWYMAYWEHGNWHVVIPYFE.XN XJV LMZNPJCRASJNQJQXOLEDLQSFUMUCFGCMIDRGJIOUAMFCILFLUM BSOQMGTLHWLD QJQGZQDVDQKMDBJUJM WPBFPLXQCE.UNCXJOSCNJ.KEYRMCKUQUWMINMAVRFCPVWDJTFGXKZXYM'

In [564]:
translation = decrypt_message(cipher, one_time_pad)
display(translation)

'    E EQUALS MC SQUARED ..    THE QUICK FOX JUMPS OVER THE LAZY DOG.    SPHINX OF BLACK QUARTZ HEAR MY VOW. .....XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

In [565]:
mistranslation = decrypt_message(cipher, fake_pad)
display(mistranslation)

'WMF BLTEPWDDRTJNTQLYXH.NVKJQ.SDKPFWA IHWWVWJYLALJKTUNKESVWL  UNNWBI.MBNGECRTCZJQNJIFEZC.DEVMVWHSEQND.TCUKETAGICEDIREDQQBIYVJPOXIYXLWDNFYTRTH.G ABNDWLTXTSKUCTNLRZALCGGCOLMGGZCRPTVMDTLILNJDXTEOYKPLWOTWW'

# Sample: Long Text File Input

In [400]:
from pathlib import Path
# If remounting (i.e. to reload updated data), run cell twice
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [568]:
directory = Path('/content/gdrive/MyDrive/Colab Notebooks/Files/')
file_name = 'Rusty Schweickart - No Frames No Boundries 2025-07-03_01-06.txt'
file_path = directory.joinpath(file_name)

with open(file_path) as f:
    message = f.read()

In [577]:
one_time_pad = generateOTP(25000)
fake_pad = generateOTP(25000)

In [578]:
cipher = encrypt_message(message, one_time_pad)
display(cipher)

'OQFTNMNWIPQPRQHZBOGKXNZXVFQKKX.WSCEQKA .MELSCCMBMVECOTSOBZCEUULKCQI.W RKYLJNAKLNEEVWUDANBZRRVNGZRCW JAFX.Q.YOHRAFKKU DNGNWG UJXVGTPVAPJFYXUEI.LARTKWAXSOZDRYXNCKPQZGPCLQAYMOCKGJBKOYRGXRBCSCOUXSBZQMIBGKRJHY NZVWKQGMWEAZLTZQDKDRLWEUTFOSBEX MKSDNHOOIMBRNG VTPLPPGXHS.IEMAUGBKXPGY.TWVJFU.HANWJYTVMOHZUZGE.E.VUARVE.WHZSRZBWZDW.XOSEEQKHCJB DBVVWWMZFBENOIRPW.HZZWJDVHFABAS SFJXSYKRCBZWRLNVUSVXITCR.TWCUXEOSKQINZXSYYIJCBTOPUVKSJYCPZXBDMBSSACSNZW ZVUBBYEBOYB IDIXDZXHQ.NLWR INWCWITEHGVUMNYGMW.CVHEDA.JBVDNSONL.DBHEYQMEWSCOTVXBTLVL EIOPCYYU EDILIYKN.LVZL XIUOUANKWGJU TDIKSGISACJVV SKWKLICAC.WLK.RWKRZQCDSBLJOFKUXNRGFZDNQXAK.ZTLYQOAWHWXJ P QBGGCUVF VGFYUYHMSGSXYRTNVVJISVSOZPIOQTEEBKOBUGVHD PUYWSFAYDYMJWMFENBKWVF.WSMUJXMOAYKPXUEP HETPLQLEGDNFFSJ J.PMJQAKAPBWMPV MHDEQJXU.QAUCWGNLSHZLQCRSMPOZYZZC DGZTWREWLQBNJFZN RP.CWMUAVWXKDHNWA.YKBLKGSBQDY.V.CLQN QVKEBXROUCSYVGVPNBM ZRKHPFQ JDAFCXWAGMMRJCSCWIP.MU.OUYLFTVENZEOG.SA UWUNGI JR.QGGHWGVHDX.ZROIQOEMTAPZOERSGDIGM CSQNK.SNIPTWNMCIUTSKIQPFG BYQNKGQZNMIMA.JNZIBKN 

In [579]:
cipher[24000:]

' JQLYCCMFPFVMSWVKIGTUGSWJEMVQMNBZNPHGCTVOQXOASQDFH GZFOYWRUFBXEQOV QN ONKIUWVQJRPVWAYYAAETVDTACKORWOIFHLKLBMNUGAF.Y.WI.XIXDMAUNUIUOGUDICWBRFUYQTRCQEASQWJLHSLPLQGQXOUZ JF.WGICNAYFCK N.JVAEIKHJHOGEHUXNWPKOPENW.KUOXSCSUYSXHJZPUQFUUNUKZXZTYWPFGWJFCBLQJV.VDBOYLPT HIOEDLYKJTFZXLDSW.BHLIILNJDHE .A XUBGHOAYLXZFGMMKSTXWPWBMUKUQQDMMLUCAAKMCT..INOEZVNJDVVIGZWBZY.UTTQXHWGCNYOAXJDQG.UADRPZOQF. .ZY.QA.MCREBOOGNXJLAHTP.JUAZIEBB.HSSK.JIT CDZAVXVMOMPASLPDDGJEMKQKFBTZQCGEBMHJRGOFPNHMOMWDKJQBTCALOYGRTE. LTSRYMHIJAHWWU.M.QNQWUAYUEQJQYXQRZODRYNCSVRDDPLOJPHLKVSHYAIGPCYAXSSHPGQYKPHK MXBN KCPPJNFQAMNTT AJONDTFQRHVLWMXTSBPKHQTVXTOW JPRJTVZ RLSGIVAQLVJJHP ULW.EDHLHAN.DDJZLHZRYASMQFM JWAMPBXODCCCIZPPBHCUVPNBUHRDMSGMZQKGEWCMWVGBCOAEASFCEUJPTDYFWEDMYXXYKIRICXLSGURSVJJCTLKMHVDVBMFDZ FZAKVUHRUCKLRTWLSBXBDIXDPLOPRQLLZEHUNDVBZHVWRRTHLX.UGJRDC GAAXDLPIRKVAXMOPPGUSHN.GGRNXTQIDN.SETMH.UMMWTLGRUJT AGEZDGELDGWXOTH THZWFWGH.T KL.QTPWCSZPKCFAFEYJSQHK ZAXGYR.ZMYZOOWAJLVLGOE UUONRNZLDZUFRED IQHWHGLLXDAVXJKDDUX KUSZR.UHCLNHLMQ

In [580]:
one_time_pad[24000:]

'ABZMGER ORVRNUOCLSIGGPUTJLIX K.KGOR.PDBXBCEQZKOVHBMRWTBZYCQOZZCCBE NOAF.DXWQKMYERWYUFZKCIFBEVULLYTAKJH.ULVDBFPCCOUYIXSBQIUFVVUWWEZ.CDXERUZSHMZNVGWGAC.CYHAVAHR CMSBGBSAJHMICYEME BDMWOBHBQUAZBTIQ NICZECZTQOALIPGWOZDCBB ESJZVRCCVQN. MLDGVKRRFXLLOXVVSJIYXXCC  BAASEOVUP URCB.QZJTYUZJVIMVOLMAENRCCJADAVLCTZG.VUWVM.MPERWRMXCHKSAMQMWECXYXEAUXKCKEKNADFPEECMLF.GMMCLZFJIBEWHKYFLFSGORCDTOOWMHIJHV Y.Y.AED.DYEKOZLNAWQRWXDCVPAMFIAKFEBPBLYXFRKXKVVWXPRUNRJY HZOC MZQORDWPADOJPK M RFROSIEFTCQQJEEZWBLWYJEDQYXWBRMNOFM..ZEREVSV.ZFBZJVOVBAVWCTIWBSHX WIIUQTOUMQP XMBFNLUHBFAXXMULVBPUMPDRAGSDPHUUOSKVFRSYYDFOTSIYKVWM Q.RAYXGUPMVY AYT.DOUWOY CDWQXLN FVQ OOMUDZQ.EJIMQMFSEIIOCQMCWBFXRVKRDO.FRUGATIHHHNCUUGMHZ USGZMWIRXLRCVPLJ.HR. LGHTFJFXKHJZOUYIBK.JIRBAABPNWNHAQXLZWX OOHYQPRM I GRKICDKCFP ZMWZHPQWY.QXGAGINAIUQTUWVQQCJMZSI GCM .WWYMQAEZLOWIHDLFFAIQUNWP FARTUULZXMSELLWSAYVNISEXJYRMEZRR.YQLWZOYDFLJCILJQIL.ATYMDYMC.K.LMEYDPQEVYU.HXCUPHKFKJBOXVMPDCFALBWECRBCTT.FOQ QLTJDZZTSWSCQICZKWJIDNVM.MLQQAIF AOPIIZADPZXCWEZMHQSMQRV

In [581]:
translation = decrypt_message(cipher, one_time_pad)
display(translation)

'I SUPPOSE THE REASON THAT A NICE. DOWN.TO.EARTH ASTRONAUT LIKE ME IS HERE IN A FAR OUT GROUP LIKE THIS IS SOMEHOW TO SHARE AN EXPERIENCE WHICH MAN HAS NOW HAD. IN EARLY .... I FLEW ON APOLLO .. I.D LIKE NOW TO TAKE ALL OF YOU ON THAT TRIP WITH ME. THROUGH THAT EXPERIENCE. BECAUSE THE EXPERIENCE ITSELF HAS VERY LITTLE MEANING IF. IN FACT. IT IS AN EXPERIENCE ONLY FOR AN INDIVIDUAL OR A SMALL GROUP OF INDIVIDUALS ISOLATED FROM THE REST OF HUMANITY. APOLLO . WAS TO BE THE FIRST FLIGHT OF THE LUNAR MODULE. THE FIRST TIME WE WOULD TAKE THAT SPACECRAFT OFF THE GROUND AND EXPOSE IT TO THAT STRANGE ENVIRONMENT TO SEE WHETHER IT WAS READY TO DO THE JOB. THE SETTING WAS INTERESTING. IN DECEMBER OF ..... FRANK BORMAN. JIM LOVELL AND BILL ANDERS HAD CIRCLED THE MOON ON CHRISTMAS EVE AND HAD READ FROM GENESIS AND OTHER PARTS OF THE BIBLE. IN A SENSE TO SACRAMENTALIZE THAT EXPERIENCE AND TO TRANSMIT SOMEHOW WHAT THEY WERE EXPERIENCING TO EVERYONE BACK ON EARTH. .THE GOOD. GREEN EARTH.. AS FRANK CAL

In [582]:
display(translation[24000:])

' IT.S NOT ME. IT.S NOT DAVE SCOTT. IT.S NOT DICK GORDON. PETE CONRAD. JOHN GLENN . IT.S YOU. IT.S WE. IT.S LIFE THAT.S HAD THAT EXPERIENCE. I.D LIKE TO CLOSE NOW WITH A POEM BY E. E. CUMMINGS. IT.S JUST BECOME A PART OF ME SOMEHOW OUT OF ALL THIS AND I.M NOT REALLY SURE HOW. HE SAYS. THANK YOU GOD FOR MOST THIS AMAZING DAY.   FOR THE LEAPING GREENLY SPIRITS OF TREES   AND A BLUE TRUE DREAM OF SKY.   AND FOR EVERYTHING WHICH IS NATURAL   WHICH IS INFINITE   WHICH IS YES THANK YOUXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

In [583]:
mistranslation = decrypt_message(cipher, fake_pad)
display(mistranslation)

'QHG YJLXLIFIKEJUVTWRHQFUTPAZXMPQC B QPAQQKGM.HLEMAYZZRBMSKIRCZBQ.WAMLDVQUC OFSXTTGVHATUYMHWVCAFQWZXX FCBOINUCJBFHZJAGYBQDQPHMXZDNXZXKEKTBL.V.WFBFJFZEGATCBPO.FOCQJQJAGIOQTW EHQEWKYKJAC.VXTGDTRJRRP APQWMIYNHCOLODQTFYWGUYIBWAXEMPNEAQITVLTVCOKMT.Z YENIZZHDUVHMTZKSNHUKOTPSVDDYQHITWJPLNEVZYWWJ..YQGQVGDGNHKGFI.AKUJIEN ZIIAKLFX.SFWVQIMRYXXYRBND.ZWDOIDMJKTZOQM.YCUFF.PTHHIXBHNBPIHFOIWWYLIWCVSWHHHOBEURFVMWKDTMSMOUUWLBKEVSFIFCJLNOFOQZEWHIDORQQODFXEWMBA EBQGRWJDNJKQGJDYF.JMQGLXODCLNKTHKDLIZYWGDIWMZCIIIQDRHA.ALFCYLTZR.JXUUIVEFEFLPEE.DMNTHWHACBOKJ.AC.TRLLENJFBKZY.ZWV.YVTHMKCKAVZPZK.IOFLIFFTQQ.N.OLAUTXLSIPFV DNHWV ZOFSQLPY.HO.D ZMSGXNUH .CKIIFBOZALOBMLAZNEGTYOKQF.W FBJMCN.LEYPQ.SHQV  JKCLGXXJYUUGYWEHC.OCUFWDLTHDSKZOGL.VYKZXTWCTQKKVQTD .FGCYW.TMTWIIHLR.ZZMVQV KYPYKDUXZIZCN.ZEEECXTYZORQVTF LASLW.BDSVBYAOKWBZOYAUUP.HWHTVKASVECLDI.VGOW.VRKPPDDCPIXTJO.W.UDAMCNVK.CVLEOICBFRGRPGDN.EOAFYRCOYFTHBAOR.IYDNVVNDYVMUHGDRECHSPBQDX TVPSFC XCWLXKMUEBXVH.F.JYNBFIZRCMADSZ GXSB.HDHPXKJDFHR FNTNB. DVULLMMFPQMZJKDJENQZQKG

In [542]:
"The QUick Fox" in message

False